Huggingface transformers 설치 및 환경 구성
터미널을 열고 아래 명령어를 입력하여 환경을 구성합니다. 명령어를 실행하지 않으면 추후 오류가 발행하므로 꼭 실행해주세요.

```
$ pip uninstall transformers -y
$ pip install transformers
$ mkdir -p transformers
__
transformers                  4.35.2
__

python -c "from transformers import pipeline; print(pipeline('sentiment-analysis')('I love you'))"
```

잘 실행되시나요? 위 코드는 10가지 GLUE task 중 감정분석을 수행하는 예제 코드입니다. 명령어 실행 결과를 보니 ‘I love you’ 문장이 99%의 확률로 positive 라고 판단했군요.

이번 노드에서는 GLUE의 'mrpc' task를 나만의 커스텀 프로젝트로 구성해서 해결해볼 예정입니다. 이 과정을 통해 Huggingface framework에 대해 좀 더 명확하게 이해하실 수 있을 것입니다.

```
pip uninstall torch -y 
pip install torch==2.0.0
```


잘 실행되시나요? 위 코드는 10가지 GLUE task 중 감정분석을 수행하는 예제 코드입니다. 명령어 실행 결과를 보니 ‘I love you’ 문장이 99%의 확률로 positive 라고 판단했군요.

이번 노드에서는 GLUE의 'mrpc' task를 나만의 커스텀 프로젝트로 구성해서 해결해볼 예정입니다. 이 과정을 통해 Huggingface framework에 대해 좀 더 명확하게 이해하실 수 있을 것입니다.

# 23-3. 커스텀 프로젝트 제작 (1) Datasets
Huggingface dataset에서 불러오기    
Huggingface dataset에는 상당한 양의 데이터가 있습니다. datasets을 이용하면 손쉽게 데이터를 불러올 수 있습니다.

```
pip uninstall huggingface-hub -y
pip install huggingface-hub==0.19.4
pip install datasets==1.15.1
```

In [2]:
from datasets import load_dataset

huggingface_mrpc_dataset = load_dataset('glue', 'mrpc')
print(huggingface_mrpc_dataset)

Reusing dataset glue (/aiffel/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


In [3]:
!pip list | grep huggingface-hub
!pip list | grep datasets

huggingface-hub               0.0.19
datasets                      1.14.0
tensorflow-datasets           4.4.0


huggingface mrpc dataset을 확인해보면 위와 같이 구성되어 있습니다.

Dataset dictionary안에 train dataset, validation dataset, test dataset으로 구성되어 있고 각 Dataset은 ‘sentence1’, ‘sentence2’, ‘label’, ‘idx’(인덱스)로 구성되어 있습니다. 해당 내용처럼 Huggingface datasets를 사용하면 손쉽게 모델의 input으로 사용할 수 있다는 장점이 있습니다.

In [4]:
train = huggingface_mrpc_dataset['train']
cols = train.column_names
cols

['sentence1', 'sentence2', 'label', 'idx']

In [5]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

sentence1 : Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
sentence2 : Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .
label : 1
idx : 0


sentence1 : Yucaipa owned Dominick 's before selling the chain to Safeway in 1998 for $ 2.5 billion .
sentence2 : Yucaipa bought Dominick 's in 1995 for $ 693 million and sold it to Safeway for $ 1.8 billion in 1998 .
label : 0
idx : 1


sentence1 : They had published an advertisement on the Internet on June 10 , offering the cargo for sale , he added .
sentence2 : On June 10 , the ship 's owners had published an advertisement on the Internet , offering the explosives for sale .
label : 1
idx : 2


sentence1 : Around 0335 GMT , Tab shares were up 19 cents , or 4.4 % , at A $ 4.56 , having earlier set a record high of A $ 4.57 .
sentence2 : Tab shares jumped 20 cents , or 4.6 % , to set a record closing high at A $ 4.57 .
label : 0

- Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .
- Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .

두 문장의 순서만 도치되었을 뿐, 의미가 같기에 label : 1로 동일한 문장이라 평가할 수 있겠네요!

# Custom Dataset 만들기
- Huggingface에 원하는 데이터셋이 없다면 어떡하죠? 걱정마세요, Huggingface datasets API를 활용하면 데이터셋을 직접 만들 수도 있고, 업로드까지도 가능합니다!

- 이번에는 MRPC 데이터셋을 직접 만들어보도록 하겠습니다. GLUE 데이터셋은 홈페이지에서도 원본을 다운로드할 수도 있지만, tensorflow_datasets에서 불러와 Huggingface dataset에 맞게 가공해봅시다.

In [6]:
import tensorflow_datasets as tfds
from datasets import Dataset

tf_dataset, tf_dataset_info = tfds.load('glue/mrpc', with_info=True)

In [7]:
tf_dataset['train']

<PrefetchDataset shapes: {idx: (), label: (), sentence1: (), sentence2: ()}, types: {idx: tf.int32, label: tf.int64, sentence1: tf.string, sentence2: tf.string}>

- What is up with ".take" method? 
    - How should I use it freely?

In [8]:
examples = tf_dataset['train'].take(5)

for example in examples:
    for col in cols:
        print(col, ":", example[col])
    print('\n')

sentence1 : tf.Tensor(b'The identical rovers will act as robotic geologists , searching for evidence of past water .', shape=(), dtype=string)
sentence2 : tf.Tensor(b'The rovers act as robotic geologists , moving on six wheels .', shape=(), dtype=string)
label : tf.Tensor(0, shape=(), dtype=int64)
idx : tf.Tensor(1680, shape=(), dtype=int32)


sentence1 : tf.Tensor(b"Less than 20 percent of Boise 's sales would come from making lumber and paper after the OfficeMax purchase is completed .", shape=(), dtype=string)
sentence2 : tf.Tensor(b"Less than 20 percent of Boise 's sales would come from making lumber and paper after the OfficeMax purchase is complete , assuming those businesses aren 't sold .", shape=(), dtype=string)
label : tf.Tensor(0, shape=(), dtype=int64)
idx : tf.Tensor(1456, shape=(), dtype=int32)


sentence1 : tf.Tensor(b'Spider-Man snatched $ 114.7 million in its debut last year and went on to capture $ 403.7 million .', shape=(), dtype=string)
sentence2 : tf.Tensor(b'Spi

Huggingface dataset과의 큰 차이는 없어보입니다. 다만 데이터가 tf.Tensor형태로 묶여있다는 차이점을 확인할 수 있습니다.

Huggingface dataset가 이중 딕셔너리 내부에 데이터를 리스트 형태로 담았기에, 이와 같은 방식으로 tf dataset을 재구성합니다.

In [9]:
# Tensorflow dataset 구조를 python dict 형식으로 변경
# Dataset이 train, validation, test로 나뉘도록 구성
train_dataset = tfds.as_dataframe(tf_dataset['train'], tf_dataset_info)
val_dataset = tfds.as_dataframe(tf_dataset['validation'], tf_dataset_info)
test_dataset = tfds.as_dataframe(tf_dataset['test'], tf_dataset_info)

In [10]:
train_dataset.head(2)

,idx,label,sentence1,sentence2
0,1680,0,b'The identical rovers will act as robotic geo...,"b'The rovers act as robotic geologists , movin..."
1,1456,0,"b""Less than 20 percent of Boise 's sales would...","b""Less than 20 percent of Boise 's sales would..."


In [11]:
# dataframe 데이터를 dict 내부에 list로 변경
train_dataset = train_dataset.to_dict('list')
val_dataset = val_dataset.to_dict('list')
test_dataset = test_dataset.to_dict('list')

In [12]:
# Huggingface dataset
tf_train_dataset = Dataset.from_dict(train_dataset)
tf_val_dataset = Dataset.from_dict(val_dataset)
tf_test_dataset = Dataset.from_dict(test_dataset)

In [16]:
tf_train_dataset

Dataset({
    features: ['idx', 'label', 'sentence1', 'sentence2'],
    num_rows: 3668
})

이렇게 우리는 같은 MRPC 데이터셋을

Huggingface datasets에서 불러도 와보고,   
외부에서 데이터를 가져와 커스터마이징도 해 보았습니다!

## Huggingface Auto Classes를 이용하는 방법
데이터셋 커스터마이징 작업을 잘 진행했다면 이미 절반 이상 진행한 것이나 마찬가지입니다. NLP 모델링의 핵심을 이루는 Tokenizer와 Model은 framework에서 이미 잘 만들어져 있는 것을 쉽게 가져다 쓸 수 있기 때문입니다.

이번엔 Huggingface에서 Auto Model과 이에 상응하는 tokenizer을 불러오겠습니다.

In [13]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
huggingface_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

Huggingface의 경우 AutoTokenizer, AutoModel기능을 지원합니다.

AutoTokenizer와 AutoModel은 Huggingface에서 지원하는 Auto Class입니다.

Auto class는 from_pretrained 메소드를 이용해 pretrained model의 경로 혹은 이름만 안다면 자동으로 생성하는 방법입니다.

즉 bert를 사용할때 BertTokenizer, RoBERTa를 사용할때 RobertaTokenizer를 사용하게 되는데 AutoTokenizer를 이용하면 자동으로 BERT모델은 BERT로 RoBERTa모델은 RoBERTa로 바꿔줍니다.

model도 마찬가지입니다. 다만 model의 경우 AutoModel을 그대로 사용하기보다 특정 task를 지정하는 방식인 AutoModelForSequenceClassification을 사용하는걸 권장드립니다.

Auto class는 다양한 모델에 자동으로 맞출 수 있기 떄문에 특정 task와 dataset이 주어져있는 경우 모델을 다양하게 넣어 실험할 수 있습니다.

그렇기에 Auto class를 유용하게 활용하는 것을 추천합니다.

Tokenizer와 Model을 만들었다면 이제 토크나이징하는 방법을 알아보도록 하겠습니다.

토크나이징은 transform이라는 함수를 만들고 이전에 만들어두었던 Tokenizer를 사용하는데 이때 dataset의 형태를 확인하고 바꿀 대상을 지정해야 합니다.

mrpc의 경우 sentence1, sentence2가 토크나이징할 대상이므로 data[’sentence1’], data[’sentence2’]로 인덱싱해서 지정합니다.

truncation은 특정 문장이 길어 모델을 다루기 힘들어 질 수 있으므로 짧게 자르는 것을 의미합니다.

return_token_type_ids는 문장이 한개이상일 때 나뉘는걸 보여줍니다. (해당 내용은 task에 필요없으므로 제거합니다)

예시까지 확인하면 다음과 같이 확인할 수 있습니다.

huggingface_mrpc_dataset 의 train dataset 중 5개의 샘플을 불러와 토크나이저를 적용해보고, 그 결과를 출력해볼까요?

In [14]:
def transform(data):
    return huggingface_tokenizer(
        data['sentence1'],
        data['sentence2'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )

# Q. sample 5개를 불러와 tokenizer의 결과를 출력해주세요
# [[YOUR CODE]]

### How to shuffle this dataset object 

In [15]:
tf_train_dataset

Dataset({
    features: ['idx', 'label', 'sentence1', 'sentence2'],
    num_rows: 3668
})

In [16]:
num_samples = len(tf_train_dataset)

In [17]:
import random

In [18]:
# Generate a list of 5 random indices
indices = random.sample(range(num_samples), 5)

In [19]:
# Select the subset of the dataset
subset = tf_train_dataset.select(indices)

In [20]:
hf_dataset = huggingface_mrpc_dataset.map(transform, batched=True)

# train & validation & test split
hf_train_dataset = hf_dataset['train']
hf_val_dataset = hf_dataset['validation']
hf_test_dataset = hf_dataset['test']

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [21]:
# Q. tf_train_dataset에 transform 함수를 매핑해봅시다. 어떤 오류가 발생하나요?
# [[YOUR CODE]]
tf_train_dataset.map(transform, batched=True)
# 오류 발생

  0%|          | 0/4 [00:00<?, ?ba/s]

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

보아하니 데이터 타입이 맞지 않아 발생하는 오류로 보입니다. 이 경우 기존 transform 함수가 sentence를 불러올 때 디코딩을 해주면 해결됩니다.

새로운 함수 transform_df 를 아래와 같이 정의하고 커스텀 데이터셋에 적용해보겠습니다.

In [22]:
def transform_tf(batch):
    sentence1 = [s.decode('utf-8') for s in batch['sentence1']]
    sentence2 = [s.decode('utf-8') for s in batch['sentence2']]
    return huggingface_tokenizer(
        sentence1,
        sentence2,
        truncation=True,
        padding='max_length',
        return_token_type_ids=False,
    )

# 토큰화 및 패딩을 적용
tf_train_dataset = tf_train_dataset.map(transform_tf, batched=True)
tf_val_dataset = tf_val_dataset.map(transform_tf, batched=True)
tf_test_dataset = tf_test_dataset.map(transform_tf, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

# 3-5. 커스텀 프로젝트 제작 (3) Train/Evaluation과 Test

> Trainer를 활용한 학습   
이제 다 왔습니다! Huggingface의 Trainer을 활용해 학습을 진행해보도록 하겠습니다.

- Trainer를 사용하기 위해서는 TrainingArguments를 통해 학습 관련 설정을 미리 지정해야 합니다.

In [23]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 16,   # 각 device 당 batch size
    per_device_eval_batch_size = 16,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

아래에서 생성하게 될 Trainer의 인자로 넘겨주어야 할 것 중에 compute_metrics 메소드가 있습니다.

이것은 task가 classification인지 regression인지에 따라 모델의 출력 형태가 달라지므로 task별로 적합한 출력 형식을 고려해 모델의 성능을 계산하는 방법을 미리 지정해 두는 것입니다.

MRPC 데이터셋은 binary classification에 해당하겠죠?

In [24]:
from datasets import load_metric
metric = load_metric('glue', 'mrpc')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

trainer에 model, arguments, train_dataset, eval_dataset, compute_metrics를 넣고 train을 진행합니다.

In [30]:
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()
print("슝~")

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 690


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.393247,0.818627,0.871972
2,No log,0.400277,0.855392,0.902801
3,0.432900,0.408132,0.860294,0.903226


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16


Training complet

슝~


In [31]:
trainer.evaluate(hf_test_dataset)

print("완료")

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1.
***** Running Evaluation *****
  Num examples = 1725
  Batch size = 16


완료


In [25]:
import time

def measure_time(func):
    # This is a decorator function that measures the execution time of another function
    def wrapper(*args, **kwargs):
        # Get the current time before calling the function
        start = time.time()
        # Call the function with the given arguments
        result = func(*args, **kwargs)
        # Get the current time after calling the function
        end = time.time()
        # Calculate the elapsed time
        elapsed = end - start
        # Print the elapsed time
        print(f"{func.__name__} took {elapsed} seconds to execute.")
        # Return the result of the function
        return result
    # Return the wrapper function
    return wrapper

In [34]:
# Use the decorator to measure the time of any function
@measure_time
def add(a, b):
    # A simple function that adds two numbers
    return a + b

# Call the function
add(10, 20)

add took 9.5367431640625e-07 seconds to execute.


30

In [35]:
#메모리를 비워줍니다.
del huggingface_model

> 방금 학습시킨 모델은 Huggingface dataset으로 학습시켰는데요, 앞에서 우리가 만든 커스텀 데이터셋 (tf_train_dataset, tf_val_dataset, tf_test_dataset)으로도 학습을 진행해보고, 결과를 비교해봅시다.

> CUDA out of memory 

In [44]:
import gc
gc.collect()

618

In [26]:
@measure_time
def train_obj():
    trainer = Trainer(
        model=huggingface_model,           # 학습시킬 model
        args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
        train_dataset=tf_train_dataset,    # training dataset
        eval_dataset=tf_val_dataset,       # evaluation dataset
        compute_metrics=compute_metrics)
    trainer.train()    
train_obj()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 690


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.496457,0.816176,0.878444
2,No log,0.416544,0.848039,0.896321
3,0.430800,0.427227,0.840686,0.889267


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16


Training complet

train_obj took 511.64282512664795 seconds to execute.


In [33]:
trainer = Trainer(
        model=huggingface_model,           # 학습시킬 model
        args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
        train_dataset=tf_train_dataset,    # training dataset
        eval_dataset=tf_val_dataset,       # evaluation dataset
        compute_metrics=compute_metrics)
trainer.train()   

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 690


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.684136,0.840686,0.893268
2,No log,0.719155,0.852941,0.899666
3,0.163800,0.690508,0.852941,0.898305


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 16


Training complet

TrainOutput(global_step=690, training_loss=0.13782282981319705, metrics={'train_runtime': 517.9657, 'train_samples_per_second': 21.245, 'train_steps_per_second': 1.332, 'total_flos': 1457671254810624.0, 'train_loss': 0.13782282981319705, 'epoch': 3.0})

In [35]:
import time

start = time.time() # get the current time in seconds
# your code here
trainer.evaluate(tf_test_dataset)

end = time.time() # get the current time in seconds
elapsed = end - start # calculate the elapsed time in seconds
print(elapsed) # print the elapsed time

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentence2, idx, sentence1.
***** Running Evaluation *****
  Num examples = 1725
  Batch size = 16
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLCriterion_updateOutput_kernel: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
/pytorch/aten/src/THCUNN/ClassNLLCriterion.cu:108: cunn_ClassNLLC

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.